I will be using the Latent Dirichlet Allocation (LDA) from Gensim package along with the Mallet’s implementation (via Gensim). Mallet has an efficient implementation of the LDA. It is known to run faster and gives better topics segregation

In [1]:
# !python -m spacy download en
# !pip install --user pyldavis
!python -m pip install --upgrade pip
!pip install --user pyLDAvis.gensim

Looking in indexes: http://pypi.douban.com/simple/


Looking in indexes: http://pypi.douban.com/simple/


ERROR: Could not find a version that satisfies the requirement pyLDAvis.gensim (from versions: none)
ERROR: No matching distribution found for pyLDAvis.gensim


In [5]:
import pandas as pd
import numpy as np
import re

import nltk
nltk.download('stopwords')

from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\natha\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [6]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [7]:
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
print(df.target_names.unique())
df.head()

['rec.autos' 'comp.sys.mac.hardware' 'comp.graphics' 'sci.space'
 'talk.politics.guns' 'sci.med' 'comp.sys.ibm.pc.hardware'
 'comp.os.ms-windows.misc' 'rec.motorcycles' 'talk.religion.misc'
 'misc.forsale' 'alt.atheism' 'sci.electronics' 'comp.windows.x'
 'rec.sport.hockey' 'rec.sport.baseball' 'soc.religion.christian'
 'talk.politics.mideast' 'talk.politics.misc' 'sci.crypt']


,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space


In [8]:
# Convert to list
data = df.content.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['From: (wheres my thing) Subject: WHAT car is this!? Nntp-Posting-Host: '
 'rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: '
 '15 I was wondering if anyone out there could enlighten me on this car I saw '
 'the other day. It was a 2-door sports car, looked to be from the late 60s/ '
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition, the front bumper was separate from the rest of the body. This is '
 'all I know. If anyone can tellme a model name, engine specs, years of '
 'production, where this car is made, history, or whatever info you have on '
 'this funky looking car, please e-mail. Thanks, - IL ---- brought to you by '
 'your neighborhood Lerxst ---- ']


## Divide the rows into sentences with gensim's simple_preprocess and remove punctuations

In [9]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]


In [10]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting_host', 'rac_wam_umd_edu', 'organization', 'university', 'of', 'maryland_college_park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front_bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']


In [12]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [15]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# !python -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['where', 's', 'thing', 'car', 'nntp_poste', 'host', 'rac_wam', 'umd', 'organization', 'university', 'park', 'line', 'wonder', 'enlighten', 'car', 'saw', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'bricklin', 'door', 'really', 'small', 'addition', 'separate', 'rest', 'body', 'know', 'tellme', 'model', 'name', 'engine', 'spec', 'year', 'production', 'car', 'make', 'history', 'info', 'funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst']]


In [16]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 5), (6, 1), (7, 2), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 2), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1)]]


In [18]:
id2word[0]
id2word[3]

'bring'

In [20]:
# Human readable format of corpus (term-frequency)
[[(f'id: {id}', id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('id: 0', 'addition', 1),
  ('id: 1', 'body', 1),
  ('id: 2', 'bricklin', 1),
  ('id: 3', 'bring', 1),
  ('id: 4', 'call', 1),
  ('id: 5', 'car', 5),
  ('id: 6', 'day', 1),
  ('id: 7', 'door', 2),
  ('id: 8', 'early', 1),
  ('id: 9', 'engine', 1),
  ('id: 10', 'enlighten', 1),
  ('id: 11', 'funky', 1),
  ('id: 12', 'history', 1),
  ('id: 13', 'host', 1),
  ('id: 14', 'info', 1),
  ('id: 15', 'know', 1),
  ('id: 16', 'late', 1),
  ('id: 17', 'lerxst', 1),
  ('id: 18', 'line', 1),
  ('id: 19', 'look', 2),
  ('id: 20', 'mail', 1),
  ('id: 21', 'make', 1),
  ('id: 22', 'model', 1),
  ('id: 23', 'name', 1),
  ('id: 24', 'neighborhood', 1),
  ('id: 25', 'nntp_poste', 1),
  ('id: 26', 'organization', 1),
  ('id: 27', 'park', 1),
  ('id: 28', 'production', 1),
  ('id: 29', 'rac_wam', 1),
  ('id: 30', 'really', 1),
  ('id: 31', 'rest', 1),
  ('id: 32', 's', 1),
  ('id: 33', 'saw', 1),
  ('id: 34', 'separate', 1),
  ('id: 35', 'small', 1),
  ('id: 36', 'spec', 1),
  ('id: 37', 'sport', 1),
  (

In [21]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [22]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.538*"ax" + 0.083*"_" + 0.043*"max" + 0.023*"internet" + 0.011*"family" + '
  '0.007*"iran" + 0.007*"fbi" + 0.006*"ai" + 0.006*"intel" + 0.006*"mr"'),
 (1,
  '0.067*"proof" + 0.059*"gateway" + 0.055*"michigan" + 0.015*"cooperation" + '
  '0.009*"n" + 0.008*"cord" + 0.006*"wipe" + 0.005*"in" + 0.005*"declaration" '
  '+ 0.005*"bk"'),
 (2,
  '0.027*"do" + 0.016*"say" + 0.016*"make" + 0.015*"think" + 0.014*"go" + '
  '0.014*"know" + 0.013*"time" + 0.012*"well" + 0.012*"see" + 0.012*"people"'),
 (3,
  '0.029*"state" + 0.025*"law" + 0.020*"people" + 0.020*"issue" + '
  '0.019*"government" + 0.018*"right" + 0.016*"public" + 0.015*"child" + '
  '0.013*"accept" + 0.012*"person"'),
 (4,
  '0.139*"physical" + 0.056*"direct" + 0.056*"robert" + 0.032*"rob" + '
  '0.026*"adam" + 0.025*"daughter" + 0.020*"human_being" + 0.013*"ho" + '
  '0.010*"cmu" + 0.009*"sister"'),
 (5,
  '0.061*"season" + 0.047*"nhl" + 0.047*"goal" + 0.044*"scott" + 0.043*"wing" '
  '+ 0.034*"pen" + 0.030*"van" + 0.029

## Topic Coherence Score:

In [23]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -14.273586566239727

Coherence Score:  0.5137725268806684


In [25]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

C:\Users\natha\AppData\Roaming\Python\Python38\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.358189  0.017463       1        1  35.056378
8      0.282085 -0.220944       2        1  16.038017
14     0.294960 -0.081829       3        1  14.324550
0      0.003513 -0.050851       4        1   6.009609
3      0.186894  0.225524       5        1   5.584453
7      0.094503 -0.166008       6        1   4.229497
15     0.187580  0.245032       7        1   4.160920
6      0.080533  0.111892       8        1   3.849918
10     0.011231 -0.100317       9        1   2.105075
17    -0.069537 -0.050278      10        1   1.578073
12    -0.049271  0.110786      11        1   1.456550
19    -0.105393 -0.013313      12        1   1.386439
9     -0.056588 -0.045940      13        1   1.252654
11    -0.107564  0.020523      14        1   1.137261
5     -0.160923  0.000434      15        1   0.615804
18    -0.159995 -0.002578      16        1   0.440360
16    -0.190495 -0.001131      17        1   0.273142
4     -0.191266 -0.000082      18        1   0.244990
13    -0.204291  0.000813      19        1   0.150895
1     -0.204166  0.000804      20        1   0.105415, topic_info=              Term          Freq         Total Category  logprob  loglift
5422            ax  49570.000000  49570.000000  Default  30.0000  30.0000
18            line  18756.000000  18756.000000  Default  29.0000  29.0000
26    organization  17206.000000  17206.000000  Default  28.0000  28.0000
1086             _   7674.000000   7674.000000  Default  27.0000  27.0000
232          write  14416.000000  14416.000000  Default  26.0000  26.0000
...            ...           ...           ...      ...      ...      ...
6677    consulting      0.017292      1.241361  Topic20 -11.4454   2.5813
6788     connected      0.017292      1.241362  Topic20 -11.4454   2.5813
4658       connect      0.017293      1.241407  Topic20 -11.4454   2.5813
6517    connection      0.017292      1.241394  Topic20 -11.4454   2.5813
6195         laugh      0.017292      1.241388  Topic20 -11.4454   2.5813

[875 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
1086      4  0.999908             _
1589      5  0.998664        accept
95        3  0.999091        access
3756     16  0.980767  access_digex
1779      5  0.734031           act
...     ...       ...           ...
45        9  0.247415          year
45       11  0.001371          year
45       13  0.023703          year
4430     18  0.942316          york
2608     19  0.989301          zone

[1078 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 9, 15, 1, 4, 8, 16, 7, 11, 18, 13, 20, 10, 12, 6, 19, 17, 5, 14, 2])